In [7]:
import os
import json
import urllib.request
import pandas as pd
import numpy as np
import re

# -----------------------------
# Configuration
# -----------------------------
DATA_DIR = "hpo_data"
os.makedirs(DATA_DIR, exist_ok=True)

PHENOTYPE_HPOA_URL = (
    "https://github.com/obophenotype/human-phenotype-ontology/"
    "releases/latest/download/phenotype.hpoa"
)

HP_JSON_URL = (
    "https://github.com/obophenotype/human-phenotype-ontology/"
    "releases/latest/download/hp.json"
)

GENES_TO_DISEASE_URL = (
    "https://github.com/obophenotype/human-phenotype-ontology/"
    "releases/latest/download/genes_to_disease.txt"
)

CLINVAR_XLSX = "clinvar conditions.xlsx"
OMIM_COL = "OMIM"
ORPHA_COL = "Orphanet"

# # -----------------------------
# # Load OMIM IDs from ClinVar Excel
# # -----------------------------
# clinvar_df = pd.read_excel(CLINVAR_XLSX)

# # Extract OMIM IDs, drop missing values
# omim_series = (
#     clinvar_df[OMIM_COLUMN_NAME]
#     .dropna()
#     .astype(str)
# )

# # Normalize OMIM format
# def normalize_omim(x):
#     x = x.strip()
#     if x.isdigit():
#         return f"OMIM:{x}"
#     if x.startswith("OMIM:"):
#         return x
#     return None

# OMIM_IDS = (
#     omim_series
#     .apply(normalize_omim)
#     .dropna()
#     .unique()
#     .tolist()
# )

# -----------------------------
# Load OMIM + Orphanet IDs from Excel
# -----------------------------
clinvar_df = pd.read_excel(CLINVAR_XLSX)

def normalize_omim(x):
    x = str(x).strip()
    if x.isdigit():
        return f"OMIM:{x}"
    if x.startswith("OMIM:"):
        return x
    return None

def normalize_orpha(x):
    x = str(x).strip()
    if x.isdigit():
        return f"ORPHA:{x}"
    if x.upper().startswith("ORPHA:"):
        return x.upper()
    return None

omim_ids = (
    clinvar_df[OMIM_COL]
    .dropna()
    .astype(str)
    .str.split(r"[;,|]")
    .explode()
    .apply(normalize_omim)
    .dropna()
    .unique()
)

orpha_ids = (
    clinvar_df.loc[clinvar_df[OMIM_COL].isna(), ORPHA_COL]
    .dropna()
    .astype(str)
    .str.split(r"[;,|]")
    .explode()
    .apply(normalize_orpha)
    .dropna()
    .unique()
)

DISEASE_IDS = sorted(set(omim_ids).union(orpha_ids))

print(f"Loaded {len(omim_ids)} OMIM IDs")
print(f"Loaded {len(orpha_ids)} Orphanet-only IDs")
print(f"Total disease IDs: {len(DISEASE_IDS)}")

# OMIM_IDS = [
#     "OMIM:617892","OMIM:614808","OMIM:615426","OMIM:615515",
#     "OMIM:616208","OMIM:616437","OMIM:617921","OMIM:105400",
#     "OMIM:205250","OMIM:300857","OMIM:606070","OMIM:606640",
#     "OMIM:619133","OMIM:608030","OMIM:608031","OMIM:608627",
#     "OMIM:611895","OMIM:612069","OMIM:612577","OMIM:613435",
#     "OMIM:613954","OMIM:600795","OMIM:617839","OMIM:619141"
# ]

def download(url, out_path):
    if not os.path.exists(out_path):
        print(f"Downloading {os.path.basename(out_path)}")
        urllib.request.urlretrieve(url, out_path)
    else:
        print(f"{os.path.basename(out_path)} already exists, skipping download.")

# -----------------------------
# Step 1: Download files
# -----------------------------
phenotype_hpoa_path = os.path.join(DATA_DIR, "phenotype.hpoa")
hp_json_path = os.path.join(DATA_DIR, "hp.json")
genes_to_disease_path = os.path.join(DATA_DIR, "genes_to_disease.txt")

download(PHENOTYPE_HPOA_URL, phenotype_hpoa_path)
download(HP_JSON_URL, hp_json_path)
download(GENES_TO_DISEASE_URL, genes_to_disease_path)


# process the json frequency entries to get numerical values
with open(os.path.join(DATA_DIR, "hp.json")) as f:
    hp = json.load(f)

hp_freq_map = {}

for term in hp["graphs"][0]["nodes"]:
    term_id = term["id"]  # e.g., 'http://purl.obolibrary.org/obo/HP_0040283'
    # Extract just the short HP ID
    hp_id = term_id.split("_")[-1]  # '0040283'
    
    # Look for definition text
    definition = term.get("meta", {}).get("definition", {}).get("val", "")
    
    # Extract percentages using regex
    percents = [float(x) for x in re.findall(r"(\d+\.?\d*)%", definition)]
    
    if len(percents) == 2:
        # Use midpoint of the range
        hp_freq_map[f"HP:{hp_id}"] = np.mean(percents)
    elif len(percents) == 1:
        hp_freq_map[f"HP:{hp_id}"] = percents[0]

# -----------------------------
# Step 2: Load phenotype.hpoa
# -----------------------------
hpoa = pd.read_csv(
    phenotype_hpoa_path,
    sep="\t",
    comment="#",
    header=None,
    names=[
        "OMIM_ID","DiseaseName","Qualifier","HP_ID",
        "Reference","Evidence","Onset","Frequency",
        "Sex","Modifier","Aspect","Biocuration"
    ]
)

# Filter for the ALS OMIM IDs
hpoa = hpoa[hpoa["OMIM_ID"].isin(DISEASE_IDS)]
print(f"Filtered phenotype rows: {len(hpoa)}")

# print(df["Frequency"].unique())

# Process Frequency Column
def parse_frequency(freq):
    if pd.isna(freq) or freq == "":
        return pd.Series([np.nan, np.nan, np.nan])

    freq = str(freq).strip()

    # Fraction case
    if "/" in freq:
        try:
            a, t = freq.split("/")
            a, t = float(a), float(t)
            return pd.Series([a, t, (a / t) * 100])
        except ValueError:
            pass

    # HP code case
    if freq.startswith("HP:") and freq in hp_freq_map:
        return pd.Series([np.nan, np.nan, hp_freq_map[freq]])

    # Fallback
    return pd.Series([np.nan, np.nan, np.nan])

# apply to hpoa
hpoa[["affected", "total", "percent"]] = hpoa["Frequency"].apply(parse_frequency)
hpoa["percent"] = hpoa["percent"].round(2)

# -----------------------------
# Step 3: Load hp.json for HP -> name mapping
# -----------------------------
with open(hp_json_path, "r") as f:
    hp_data = json.load(f)

hp_terms = {}
for node in hp_data["graphs"][0]["nodes"]:
    uri = node.get("id", "")
    label = node.get("lbl", None)
    if uri.startswith("http://purl.obolibrary.org/obo/HP_") and label:
        hp_id = uri.split("/")[-1].replace("_", ":")
        hp_terms[hp_id] = label

# Add human-readable phenotype names
hpoa["Phenotype"] = hpoa["HP_ID"].map(hp_terms)

# check hpoa
display(hpoa.head(10))

# -----------------------------
# Step 4: Load genes_to_disease.txt
# -----------------------------
genes_df = pd.read_csv(genes_to_disease_path, sep="\t")

print(genes_df.columns)
display(genes_df)

# Normalize OMIM ID format
# genes_df["OMIM_ID"] = genes_df["disease_id"].astype(str).apply(lambda x: f"OMIM:{x}")
genes_df["OMIM_ID"] = genes_df["disease_id"]

# Filter only the OMIM IDs we care about
# genes_df = genes_df[genes_df["OMIM_ID"].isin(DISEASE_IDS)]

genes_df = genes_df[genes_df["OMIM_ID"].str.startswith("OMIM:")]
genes_df = genes_df[genes_df["OMIM_ID"].isin(omim_ids)]

# Keep just gene symbol + OMIM
# genes_df = genes_df[["gene_symbol","OMIM_ID"]].rename(columns={"geneSymbol": "Gene"})
genes_df = genes_df[["gene_symbol","OMIM_ID"]]

display(genes_df)

# -----------------------------
# Step 5: Merge gene info into phenotype DataFrame
# -----------------------------
df = (
    hpoa
    .merge(genes_df, on="OMIM_ID", how="left")
    [["gene_symbol","OMIM_ID", "DiseaseName", "HP_ID","Phenotype", "affected", "total", "percent"]]
    .sort_values(["gene_symbol","OMIM_ID","HP_ID"])
    .reset_index(drop=True)
)

print(df.head())
print(f"\nTotal rows in final DataFrame: {len(df)}")

# -----------------------------
# Step 6: Save (optional)
# -----------------------------
hpoa.to_csv("als_hpoa.csv", index=False)
genes_df.to_csv("als_genes.csv", index=False)

df.to_csv("als_hpo_gene_phenotypes.csv", index=False)
display(df)

print(f"{df['percent'].isna().sum()} phenotypes have no frequency info")

# Create a boolean mask: True if Phenotype contains 'onset'
onset_mask = df["Phenotype"].str.contains("onset", case=False, na=False)

# Table with only onset-related rows
onset_df = df[onset_mask].copy()

# Table with all other clinical phenotypes
clinical_df = df[~onset_mask].copy()

# processing of onset_df for onset categories
onset_df = onset_df.copy()
# fill blank with 100
onset_df["percent"] = onset_df["percent"].fillna(100)
onset_wide = onset_df.pivot_table(
    index=["gene_symbol", "OMIM_ID", "DiseaseName"],
    columns="Phenotype",
    values="percent",
    aggfunc="max"   # or 'mean', but max is safest here
)
onset_wide = onset_wide.reset_index()
onset_wide.columns.name = None
onset_wide = onset_wide.fillna(0)

# normalize so every goes to 100
# Columns that identify a disease
id_cols = ["gene_symbol", "OMIM_ID", "DiseaseName"]

# Detect onset-category columns automatically
onset_cols = [
    c for c in onset_wide.columns
    if c not in id_cols and "onset" in c.lower()
]

# Work on a copy
onset_wide_norm = onset_wide.copy()

# Row-wise sum of onset percentages
row_sums = onset_wide_norm[onset_cols].sum(axis=1)

# Normalize only rows where total exceeds 100
mask = row_sums > 100

onset_wide_norm.loc[mask, onset_cols] = (
    onset_wide_norm.loc[mask, onset_cols]
    .div(row_sums[mask], axis=0)
    .mul(100)
)

# Optional: round for readability
onset_wide_norm[onset_cols] = onset_wide_norm[onset_cols].round(2)

# save
onset_df.to_csv("als_genes_onset.csv", index=False)
onset_wide.to_csv("als_genes_onset_categories.csv", index=False)
onset_wide_norm.to_csv("als_genes_onset_categories_norm.csv", index=False)
clinical_df.to_csv("als_genes_symptoms.csv", index=False)

print(f"Rows in onset table: {len(onset_df)}")
print(f"Rows in clinical phenotype table: {len(clinical_df)}")
print(f"Diseases in onset table: {onset_df["OMIM_ID"].nunique()}")
print(f"Diseases in clinical table: {clinical_df["OMIM_ID"].nunique()}")

# For each gene, check if all percent values are NaN
genes_all_missing = df.groupby("gene_symbol")["percent"].apply(lambda x: x.isna().all())
# Count how many genes have all NaN
num_genes_all_missing = genes_all_missing.sum()
num_genes = df["gene_symbol"].nunique()
print(f"Number of genes with completely missing percent: {num_genes_all_missing} out of {num_genes}")

# For each gene, check if all percent values are NaN
genes_all_missing = clinical_df.groupby("gene_symbol")["percent"].apply(lambda x: x.isna().all())
# Count how many genes have all NaN
num_genes_all_missing = genes_all_missing.sum()
num_genes = clinical_df["gene_symbol"].nunique()
print(f"Number of genes with completely missing percent, clinical_df: {num_genes_all_missing} out of {num_genes}")


Loaded 28 OMIM IDs
Loaded 0 Orphanet-only IDs
Total disease IDs: 28
phenotype.hpoa already exists, skipping download.
hp.json already exists, skipping download.
genes_to_disease.txt already exists, skipping download.


C:\Users\William\AppData\Local\Temp\ipykernel_36220\2012924754.py:165: DtypeWarning: Columns (2,6,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  hpoa = pd.read_csv(


Filtered phenotype rows: 527


,OMIM_ID,DiseaseName,Qualifier,HP_ID,Reference,Evidence,Onset,Frequency,Sex,Modifier,Aspect,Biocuration,affected,total,percent,Phenotype
12948,OMIM:617839,Amyotrophic lateral sclerosis 23,NaN,HP:0002015,PMID:28469040,PCS,NaN,6/12,NaN,NaN,P,HPO:probinson[2022-04-11],6.0,12.0,50.0,Dysphagia
12949,OMIM:617839,Amyotrophic lateral sclerosis 23,NaN,HP:0003596,PMID:28469040,PCS,NaN,3/12,NaN,NaN,C,HPO:probinson[2022-04-11],3.0,12.0,25.0,Middle age onset
12950,OMIM:617839,Amyotrophic lateral sclerosis 23,NaN,HP:0002398,PMID:28469040,PCS,NaN,NaN,NaN,NaN,P,HPO:probinson[2022-04-11];HPO:probinson[2022-0...,NaN,NaN,NaN,Degeneration of anterior horn cells
12951,OMIM:617839,Amyotrophic lateral sclerosis 23,NaN,HP:0000726,PMID:28469040,PCS,NaN,0/12,NaN,NaN,P,HPO:skoehler[2018-10-08];HPO:probinson[2022-04...,0.0,12.0,0.0,Dementia
12952,OMIM:617839,Amyotrophic lateral sclerosis 23,NaN,HP:0003584,PMID:28469040,PCS,NaN,9/12,NaN,NaN,C,HPO:probinson[2022-04-11],9.0,12.0,75.0,Late onset
12953,OMIM:617839,Amyotrophic lateral sclerosis 23,NaN,HP:0007354,PMID:28469040,PCS,NaN,12/12,NaN,NaN,P,HPO:skoehler[2018-10-08];HPO:probinson[2022-04...,12.0,12.0,100.0,Amyotrophic lateral sclerosis
12954,OMIM:617839,Amyotrophic lateral sclerosis 23,NaN,HP:0000006,PMID:28469040,PCS,NaN,NaN,NaN,NaN,I,HPO:skoehler[2019-04-18];HPO:probinson[2022-04...,NaN,NaN,NaN,Autosomal dominant inheritance
15921,OMIM:615424,Inclusion body myopathy with early-onset Paget...,NaN,HP:0003236,PMID:20116073,PCS,NaN,5/5,NaN,NaN,P,HPO:probinson[2022-04-11],5.0,5.0,100.0,Elevated circulating creatine kinase concentra...
15922,OMIM:615424,Inclusion body myopathy with early-onset paget...,NaN,HP:0003687,PMID:20116073,PCS,NaN,NaN,NaN,NaN,P,HPO:probinson[2022-04-11],NaN,NaN,NaN,Centrally nucleated skeletal muscle fibers
15923,OMIM:615424,Inclusion body myopathy with early-onset Paget...,NaN,HP:0003596,PMID:20116073,PCS,NaN,3/5,NaN,NaN,C,HPO:probinson[2022-04-11],3.0,5.0,60.0,Middle age onset


Index(['ncbi_gene_id', 'gene_symbol', 'association_type', 'disease_id',
       'source'],
      dtype='object')


,ncbi_gene_id,gene_symbol,association_type,disease_id,source
0,NCBIGene:64170,CARD9,MENDELIAN,OMIM:212050,ftp://ftp.ncbi.nlm.nih.gov/gene/DATA/mim2gene_...
1,NCBIGene:51256,TBC1D7,MENDELIAN,OMIM:248000,ftp://ftp.ncbi.nlm.nih.gov/gene/DATA/mim2gene_...
2,NCBIGene:28981,IFT81,MENDELIAN,OMIM:617895,ftp://ftp.ncbi.nlm.nih.gov/gene/DATA/mim2gene_...
3,NCBIGene:8216,LZTR1,MENDELIAN,OMIM:616564,ftp://ftp.ncbi.nlm.nih.gov/gene/DATA/mim2gene_...
4,NCBIGene:6505,SLC1A1,POLYGENIC,OMIM:615232,ftp://ftp.ncbi.nlm.nih.gov/gene/DATA/mim2gene_...
...,...,...,...,...,...
15778,NCBIGene:55901,THSD1,UNKNOWN,ORPHA:231160,http://www.orphadata.org/data/xml/en_product6.xml
15779,NCBIGene:7049,TGFBR3,UNKNOWN,ORPHA:231160,http://www.orphadata.org/data/xml/en_product6.xml
15780,NCBIGene:1281,COL3A1,UNKNOWN,ORPHA:231160,http://www.orphadata.org/data/xml/en_product6.xml
15781,NCBIGene:83854,ANGPTL6,UNKNOWN,ORPHA:231160,http://www.orphadata.org/data/xml/en_product6.xml


,gene_symbol,OMIM_ID
7,NEK1,OMIM:617892
174,FUS,OMIM:608030
336,TRPM7,OMIM:105500
354,ANXA11,OMIM:617839
376,C9orf72,OMIM:105550
806,TBK1,OMIM:616439
810,SQSTM1,OMIM:616437
812,PRPH,OMIM:105400
813,DCTN1,OMIM:105400
814,NEFH,OMIM:105400


  gene_symbol      OMIM_ID                                DiseaseName  \
0        ALS2  OMIM:205100  Amyotrophic lateral sclerosis 2, juvenile   
1        ALS2  OMIM:205100  Amyotrophic lateral sclerosis 2, juvenile   
2        ALS2  OMIM:205100  Amyotrophic lateral sclerosis 2, juvenile   
3        ALS2  OMIM:205100  Amyotrophic lateral sclerosis 2, juvenile   
4        ALS2  OMIM:205100  Amyotrophic lateral sclerosis 2, juvenile   

        HP_ID                        Phenotype  affected  total  percent  
0  HP:0000007  Autosomal recessive inheritance       NaN    NaN      NaN  
1  HP:0000020             Urinary incontinence       1.0    3.0    33.33  
2  HP:0000183           Tongue muscle weakness       NaN    NaN      NaN  
3  HP:0000252                     Microcephaly       2.0    3.0    66.67  
4  HP:0000639                        Nystagmus       1.0    3.0    33.33  

Total rows in final DataFrame: 575


,gene_symbol,OMIM_ID,DiseaseName,HP_ID,Phenotype,affected,total,percent
0,ALS2,OMIM:205100,"Amyotrophic lateral sclerosis 2, juvenile",HP:0000007,Autosomal recessive inheritance,NaN,NaN,NaN
1,ALS2,OMIM:205100,"Amyotrophic lateral sclerosis 2, juvenile",HP:0000020,Urinary incontinence,1.0,3.0,33.33
2,ALS2,OMIM:205100,"Amyotrophic lateral sclerosis 2, juvenile",HP:0000183,Tongue muscle weakness,NaN,NaN,NaN
3,ALS2,OMIM:205100,"Amyotrophic lateral sclerosis 2, juvenile",HP:0000252,Microcephaly,2.0,3.0,66.67
4,ALS2,OMIM:205100,"Amyotrophic lateral sclerosis 2, juvenile",HP:0000639,Nystagmus,1.0,3.0,33.33
...,...,...,...,...,...,...,...,...
570,VCP,OMIM:613954,Frontotemporal dementia and/or amyotrophic lat...,HP:0007340,Lower limb muscle weakness,1.0,9.0,11.11
571,VCP,OMIM:613954,Frontotemporal dementia and/or amyotrophic lat...,HP:0007354,Amyotrophic lateral sclerosis,NaN,NaN,NaN
572,VCP,OMIM:613954,Frontotemporal dementia and/or amyotrophic lat...,HP:0008994,Proximal lower limb muscle weakness,NaN,NaN,NaN
573,VCP,OMIM:613954,Frontotemporal dementia and/or amyotrophic lat...,HP:0008997,Proximal upper limb muscle weakness,NaN,NaN,NaN


371 phenotypes have no frequency info
Rows in onset table: 48
Rows in clinical phenotype table: 527
Diseases in onset table: 24
Diseases in clinical table: 27
Number of genes with completely missing percent: 9 out of 27
Number of genes with completely missing percent, clinical_df: 9 out of 27
